Applied Data Science Capstone project - Week 5 Project

Opening a new shopping mall in Chennai- India

This project involves:
1. Building a data frame by scraping from the wikipedia page of Chennai
2. Getting the geographical coordinates of the neighbourhoods
3. Obtain the venue data for the neighbourhoods usig FourSquare API
4. Explore and cluster Neighbourhoods
5. Select the best cluster to open a shopping Mall

In [1]:
#importing necessary packages
import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
!conda install -c conda-forge geocoder --yes

import geocoder
import json 
import requests
from bs4 import BeautifulSoup 
import json
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

Scraping data from wikipedia link about Neighbourhoods in Chennai

In [2]:
#scraping data
link="https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Chennai"
data = requests.get(link).text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
# create a new DataFrame from the list
df = pd.DataFrame({"Neighborhood": neighborhoodList})

df.head()
print("The number of neighbourhoods in Chennai",df.shape)

The number of neighbourhoods in Chennai (200, 1)


In [3]:
def get_coordinates(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
coords = [ get_coordinates(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
print("The size of data frame is ",df.shape)
print (df.columns)

The size of data frame is  (200, 3)
Index(['Neighborhood', 'Latitude', 'Longitude'], dtype='object')


Create a map on Chennai along with neighbourhoods

In [4]:
add="Chennai,India"
geoloc = Nominatim(user_agent="my-application")
loc = geoloc.geocode(add)
lat = loc.latitude
long = loc.longitude
print('The geograpical coordinate of Chennai, India {}, {}.'.format(lat, long))


The geograpical coordinate of Chennai, India 13.0801721, 80.2838331.


Creating a map of Chennai

In [5]:
# create map of Toronto using latitude and longitude values
map_chennai = folium.Map(location=[lat, long], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_chennai)  
    
map_chennai

Using Four Square to explore the Neighbourhood

In [6]:
CLIENT_ID = 'P23VGT0VWRVGYOH4DC3ED4LVDX3XS4RVU3521OSPE2J1MBCW' 
CLIENT_SECRET = 'NAWSW3EVXDIPIWHV1HHQEKNQT3ZSDBJF5BMLIPID5LTHZZAO' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P23VGT0VWRVGYOH4DC3ED4LVDX3XS4RVU3521OSPE2J1MBCW
CLIENT_SECRET:NAWSW3EVXDIPIWHV1HHQEKNQT3ZSDBJF5BMLIPID5LTHZZAO


Searching for Venues with 1000 meter radius

In [7]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [8]:
# Copy the venue list ot Dataframe
venues_df = pd.DataFrame(venues)

# Column Names for data frame
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(20)


(2737, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of neighbourhoods of Chennai,13.02756,80.25386,Chamiers Cafe,13.028241,80.250240,Café
1,List of neighbourhoods of Chennai,13.02756,80.25386,Oryza Spa,13.028420,80.250366,Spa
2,List of neighbourhoods of Chennai,13.02756,80.25386,Cream Centre,13.027064,80.254961,Indian Restaurant
3,List of neighbourhoods of Chennai,13.02756,80.25386,Sandy's,13.025816,80.253910,Dessert Shop
4,List of neighbourhoods of Chennai,13.02756,80.25386,ibaco,13.028293,80.256283,Ice Cream Shop
5,List of neighbourhoods of Chennai,13.02756,80.25386,Indian Terrain,13.031155,80.251400,Men's Store
6,List of neighbourhoods of Chennai,13.02756,80.25386,Nilgiris Supermarket,13.028484,80.256301,Department Store
7,List of neighbourhoods of Chennai,13.02756,80.25386,Raintree Hotels,13.028696,80.252790,Hotel
8,List of neighbourhoods of Chennai,13.02756,80.25386,Amma nana,13.028891,80.249702,Department Store
9,List of neighbourhoods of Chennai,13.02756,80.25386,Benjarong,13.031790,80.251844,Thai Restaurant


In [9]:
#Displaying Neighbourhood by grouping categories
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adambakkam,9,9,9,9,9,9
"Adyar, Chennai",71,71,71,71,71,71
Agaram,8,8,8,8,8,8
Alandur,17,17,17,17,17,17
Alapakkam,7,7,7,7,7,7
Alwarpet,71,71,71,71,71,71
Alwarthirunagar,12,12,12,12,12,12
Aminjikarai,17,17,17,17,17,17
Amullaivoyal,2,2,2,2,2,2


In [10]:
print('The number of unique category of venues are {}.'.format(len(venues_df['VenueCategory'].unique())))

The number of unique category of venues are 192.


In [11]:
# print out the list of categories
print("The unique venues are:")
venues_df['VenueCategory'].unique()[:50]

The unique venues are:


array(['Café', 'Spa', 'Indian Restaurant', 'Dessert Shop',
       'Ice Cream Shop', "Men's Store", 'Department Store', 'Hotel',
       'Thai Restaurant', 'Breakfast Spot', 'Italian Restaurant',
       'Bakery', 'Japanese Restaurant', 'South Indian Restaurant',
       'Sandwich Place', 'Restaurant', 'Vegetarian / Vegan Restaurant',
       'Bar', 'Snack Place', 'Salon / Barbershop', 'Chinese Restaurant',
       'Modern European Restaurant', 'Coffee Shop', 'Juice Bar', 'Arcade',
       'Gym / Fitness Center', 'Tea Room', 'Nightclub', 'Yoga Studio',
       'Train Station', 'Pizza Place', 'Rock Club', 'Farmers Market',
       'Metro Station', 'Supermarket', "Women's Store",
       'North Indian Restaurant', 'Bookstore', 'Fast Food Restaurant',
       'Middle Eastern Restaurant', 'Clothing Store', 'Asian Restaurant',
       'Grocery Store', 'Movie Theater', 'Mediterranean Restaurant',
       'Lounge', 'Comfort Food Restaurant', 'BBQ Joint',
       'Electronics Store', 'Watch Shop'], dtype=ob

Analyzing Neighbourhoods in Chennai from the prespective of Venues

In [13]:
#Preprocessing of data for anaysis - Encoding category based variables 
# one hot encoding
df_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

print(df_onehot.shape)
df_onehot.head()


(2737, 193)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Astrologer,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Library,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Currency Exchange,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music School,Music Store,Music Venue,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Rock Club,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Surf Spot,Taco Place,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Whisky Bar,Women's Store,Yoga Studio
0,List of neighbourhoods of Chennai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,List of neighbourhoods of Chennai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,List of neighbourhoods of Chennai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,List of neighbo

In [14]:
# Grouping of Neighbourhoods based on category
df_grouped = df_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(df_grouped.shape)
df_grouped

(186, 193)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Astrologer,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Library,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Currency Exchange,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kerala Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music School,Music Store,Music Venue,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Rock Club,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Supermarket,Surf Spot,Taco Place,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Whisky Bar,Women's Store,Yoga Studio
0,Adambakkam,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.111111,0.000000,0.0,0.000000,0.000000,0.222222,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0

In [15]:
len(df_grouped[df_grouped["Shopping Mall"] > 0])

10

In [17]:
#Creating a new data frame for shopping mall only
df_mall = df_grouped[["Neighborhoods","Shopping Mall"]]
df_mall.head()

,Neighborhoods,Shopping Mall
0,Adambakkam,0.0
1,"Adyar, Chennai",0.0
2,Agaram,0.0
3,Alandur,0.0
4,Alapakkam,0.0


Clustering Neighbourhood in Chennai based on Shopping Mall 

In [18]:
# Number of cluster
kclusters = 3

df_clustering = df_mall.drop(["Neighborhoods"], 1)
from sklearn.cluster import KMeans
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [19]:
# Group top ten venues based on the cluster group.
df_merged = df_mall.copy()

# add clustering labels
df_merged["Cluster Labels"] = kmeans.labels_

In [20]:
df_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
df_merged.head(20)

,Neighborhood,Shopping Mall,Cluster Labels
0,Adambakkam,0.000000,0
1,"Adyar, Chennai",0.000000,0
2,Agaram,0.000000,0
3,Alandur,0.000000,0
4,Alapakkam,0.000000,0
5,Alwarpet,0.000000,0
6,Alwarthirunagar,0.000000,0
7,Aminjikarai,0.058824,1
8,Amullaivoyal,0.000000,0
9,Anna Nagar,0.000000,0


In [21]:
#merging cluster based on each neighbourhood
df_merged_1 = df_merged.join(df.set_index("Neighborhood"), on="Neighborhood",lsuffix='_left')

print(df_merged_1.shape)
df_merged_1.head()

(186, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adambakkam,0.0,0,12.99187,80.20602
1,"Adyar, Chennai",0.0,0,13.00303,80.25187
2,Agaram,0.0,0,13.10953,80.23236
3,Alandur,0.0,0,13.00010,80.20048
4,Alapakkam,0.0,0,13.04609,80.16496


In [25]:
# sort using cluster labels
print(df_merged_1.shape)
df_merged.sort_values(["Cluster Labels"], inplace=True)
df_merged

(186, 5)


,Neighborhood,Shopping Mall,Cluster Labels
0,Adambakkam,0.000000,0
1,"Adyar, Chennai",0.000000,0
2,Agaram,0.000000,0
3,Alandur,0.000000,0
4,Alapakkam,0.000000,0
5,Alwarpet,0.000000,0
6,Alwarthirunagar,0.000000,0
8,Amullaivoyal,0.000000,0
9,Anna Nagar,0.000000,0
21,Chetpet (Chennai),0.000000,0


Visulizing the resulting cluster

In [28]:
from matplotlib import *
from matplotlib import cm
# Creating Map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged_1['Latitude'], df_merged_1['Longitude'], df_merged_1['Neighborhood'], df_merged_1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Locations falling in different clusters


In [29]:
df_merged_1.loc[df_merged_1['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adambakkam,0.0,0,12.991870,80.206020
1,"Adyar, Chennai",0.0,0,13.003030,80.251870
2,Agaram,0.0,0,13.109530,80.232360
3,Alandur,0.0,0,13.000100,80.200480
4,Alapakkam,0.0,0,13.046090,80.164960
5,Alwarpet,0.0,0,13.034470,80.253980
6,Alwarthirunagar,0.0,0,13.050550,80.183970
8,Amullaivoyal,0.0,0,13.165140,80.161155
9,Anna Nagar,0.0,0,13.083590,80.210140
10,Anna Nagar West,0.0,0,13.092740,80.202210


In [30]:
df_merged_1.loc[df_merged_1['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
7,Aminjikarai,0.058824,1,13.07139,80.22256
22,Chinnakudal,0.040000,1,13.07444,80.22167
75,Koyambedu,0.058824,1,13.06606,80.19706
170,"Thirumangalam, Chennai",0.041667,1,13.08281,80.19703


In [31]:
df_merged_1.loc[df_merged_1['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
78,"Lakshmipuram, Chennai",0.016129,2,12.98659,80.22175
156,Shenoy Nagar,0.029412,2,13.07732,80.22498
162,Subramania Nagar,0.021739,2,13.04541,80.22860
163,T. Nagar,0.015625,2,13.04537,80.23390
180,Triplicane,0.016949,2,13.06296,80.27151
184,Vadapalani,0.032258,2,13.05228,80.21120
